# ConfigState example with training a model using Tensorflow

This notebook presents an example of how the config-state library can be used to design a machine learning experiment that consists in training an image classification model. We show how the different components, the dataset, the model and the optimizer can be configured and modified through a config file without requiring to write code. We also show how the experiment can be saved at regular intervals and be resumed in case of interruption.

### Requirements

The packages `tensorflow` and `tensorflow-datasets` are required for this example:
```
pip install tensorflow
pip install tensorflow-datasets
```

### The `MLExperiment` class

The `MLExperiment` class is a `ConfigState` subclass that defines the experiment consisting of training a machine learning model for image classification. It is composed of nested `ConfigState` objects that represent the different components such as `Dataset`, `Model` and `Optimizer`.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # reduce tensorflow's verbosity

from examples.tensorflow.experiment import MLExperiment

### Configuring a `MLExperiment` experiment
The directory `examples/tensorflow/configs` contains examples of configuration files that can be used to configure an experiment. Let's load one:

In [2]:
import yaml

config = yaml.load(open("tensorflow/configs/mlp.yml", 'r'), Loader=yaml.FullLoader)

An experiment can be instantiated using this configuration:

In [3]:
experiment = MLExperiment(config)

print(experiment.config_summary())

dataset:
  batch_size: 32
  name: mnist
model:
  class: MultiLayerPerceptron
  dropout_rate: 0.2
  input_shape: (28, 28, 1)
  output_units: 10
  structure: [128]
optimizer:
  class: RMSprop
  epsilon: 1e-07
  learning_rate: 0.001
  momentum: 0.0
  rho: 0.9



We can start training the model for a given number of epochs:

In [4]:
experiment.run(epochs=2)

Training for 2 epochs...
Epoch 1/2
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3002 - accuracy: 0.9116 - val_loss: 0.1469 - val_accuracy: 0.9570

### Saving and restoring an experiment

The current experiment's state can been saved into file:

In [5]:
from config_state import Serializer
import tempfile
from pathlib import Path

# create a temporary directory
temp_dir = tempfile.TemporaryDirectory()

# file that will store the experiment
file_path = Path(temp_dir.name) / 'exp.save'

# save the experiment using the Pickle serializer
Serializer({'class': 'Pickle'}).save(experiment, file_path)

The experiment can be restored and resumed:

In [6]:
experiment = Serializer({'class': 'Pickle'}).load(file_path)

experiment.run(epochs=2)

temp_dir.cleanup()

Training for 2 epochs...
Epoch 3/4
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1288 - accuracy: 0.9643 - val_loss: 0.1109 - val_accuracy: 0.9708

### Configuring a new experiment

We can customize the config dictionary to design a new experiment with a different datatet, model or optimizer:

In [7]:
config['dataset'] = {
    'name': 'cifar10' # https://www.tensorflow.org/datasets/catalog/overview#image_classification
}
config['model'] = {
    'class': 'CNN',
    'structure': [32, 'max', 64, 'max', 64]
}
config['optimizer'] = {
    'class': 'Adam',
    'learning_rate': 0.001
}

experiment = MLExperiment(config)

print(experiment.config_summary())

experiment.run(epochs=20)

dataset:
  batch_size: 32
  name: cifar10
model:
  class: CNN
  input_shape: (32, 32, 3)
  output_units: 10
  structure: [32, max, 64, max, 64]
optimizer:
  beta_1: 0.9
  beta_2: 0.9999
  class: Adam
  epsilon: 1e-07
  learning_rate: 0.001

Training for 20 epochs...
Epoch 1/20
1563/1563 [==============================] - 6s 3ms/step - loss: 1.5146 - accuracy: 0.4515 - val_loss: 1.2677 - val_accuracy: 0.5455

### ConfigState objects composability

`ConfigState` is convenient for compositing objects. For instance we can nest a `Model` into another `Ensembler` model:

In [8]:
cnn_model = {
    'class': 'CNN',
    'structure': [32, 'max', 64, 'max', 64]
}

config['model'] = {
    'class': 'Ensembler',
    'model': cnn_model,
    'ensemble_size': 4
}

config['dataset'] = {
    'name': 'cifar10',
    'batch_size': 128 # We augment the batch_size so that each ensembled models train on batches of 32 elements
}

experiment = MLExperiment(config)

print(experiment.config_summary())

print(experiment.model.keras_model.summary())


dataset:
  batch_size: 128
  name: cifar10
model:
  class: Ensembler
  ensemble_size: 4
  input_shape: (32, 32, 3)
  model:
    class: CNN
    input_shape: (32, 32, 3)
    output_units: 10
    structure: [32, max, 64, max, 64]
  output_units: 10
optimizer:
  beta_1: 0.9
  beta_2: 0.9999
  class: Adam
  epsilon: 1e-07
  learning_rate: 0.001

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 [(None, 32, 32, 3),  0           input_6[0][0]                    
__________________________________________________________________________________________________
sequential_3 (Sequential)       (None, 10)      

In [9]:
experiment.run(epochs=20)

Training for 20 epochs...
Epoch 1/20
391/391 [==============================] - 4s 8ms/step - loss: 1.8494 - accuracy: 0.3223 - val_loss: 1.5611 - val_accuracy: 0.4410

Since `Ensembler` is itself a `Model`, we can compose it into another `Ensemble` such that we can define models that are ensemble of ensemble:

In [10]:
cnn_model = {
    'class': 'CNN',
    'structure': [32, 'max', 64, 'max', 64]
}

ensemble = {
    'class': 'Ensembler',
    'model': cnn_model,
    'ensemble_size': 4
}

config['model'] = {
    'class': 'Ensembler',
    'model': ensemble,
    'ensemble_size': 4
}

config['dataset'] = {
    'name': 'cifar10',
    'batch_size': 512
}

experiment = MLExperiment(config)

# ensemble_ensemble_exp.model.model.output_units
print(experiment.config_summary())

print(experiment.model.keras_model.summary())


dataset:
  batch_size: 512
  name: cifar10
model:
  class: Ensembler
  ensemble_size: 4
  input_shape: (32, 32, 3)
  model:
    class: Ensembler
    ensemble_size: 4
    input_shape: (32, 32, 3)
    model:
      class: CNN
      input_shape: (32, 32, 3)
      output_units: 10
      structure: [32, max, 64, max, 64]
    output_units: 10
  output_units: 10
optimizer:
  beta_1: 0.9
  beta_2: 0.9999
  class: Adam
  epsilon: 1e-07
  learning_rate: 0.001

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_27 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
lambda_10 (Lambda)              [(None, 32, 32, 3),  0           input_27[0][0]                   
__________________________________

In [11]:
experiment.run(epochs=1)

Training for 1 epochs...
98/98 [==============================] - 5s 33ms/step - loss: 2.1516 - accuracy: 0.1914 - val_loss: 1.9043 - val_accuracy: 0.3362